# Role-play or persona shift?

- get activations from formatting conversation as dialogue
- original activations of chat with the model

In [1]:
import torch
import os
import json
import sys
import numpy as np


sys.path.append('.')
sys.path.append('..')

from utils.steering_utils import ActivationSteering
from utils.probing_utils import *
from utils.inference_utils import *


INFO 08-14 18:39:54 [__init__.py:235] Automatically detected platform cuda.


In [2]:
CHAT_MODEL_NAME = "google/gemma-2-27b-it"
LAYER = 22
OUTPUT_DIR = "./results/7_dialogue_chat"
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Reformat transcripts as dialogue

In [3]:
activations_dir = "/workspace/roleplay/gemma-2-27b"
transcripts_dir = "/root/git/persona-subspace/roleplay/results/gemma-2-27b/role_vectors/transcripts"

roles = ["anxious_teenager", "deep_sea_leviathan", "medieval_bard"]

In [ ]:
chat_activations = {} # role to activation tensor of (n_layers, n_tokens, hidden_dim)
chats = {} # role to list of dicts in typical chat conversation format

for role in roles:
    convo = json.load(open(f"{transcripts_dir}/{role}.json"))
    chats[role] = convo["conversation"]
    
    chat_activation = torch.load(f"{activations_dir}/{role}.pt")
    chat_activations[role] = chat_activation


In [5]:
print(chat_activations["anxious_teenager"].shape)
print(len(chats["anxious_teenager"]))

torch.Size([46, 2623, 4608])
22


In [ ]:
# need to reformat chat conversation as a dialogue

# def chat_to_dialogue(conversation, user_name, assistant_name):
#     dialogue = ""
#     for message in conversation:
#         if message["role"] == "user":
#             dialogue += f"{user_name}: {message['content']}\n"
#         elif message["role"] == "assistant":
#             dialogue += f"{assistant_name}: {message['content']}\n"
#     return dialogue

In [6]:
# save string under "dialogue" in the original json file
dialogues = {}

for role in roles:
    convo = json.load(open(f"{transcripts_dir}/{role}.json"))
    dialogues[role] = convo["dialogue"]

# names = [("Therapist", "Christina"), ("Behemoth", "Leviathan"), ("NPC", "Bard")]

# for role, name in zip(roles, names):
#     dialogue = chat_to_dialogue(chats[role], name[0], name[1])
#     dialogues[role] = dialogue

#     original_convo = json.load(open(f"{transcripts_dir}/{role}.json"))
#     original_convo["dialogue"] = dialogue

#     with open(f"{transcripts_dir}/{role}.json", "w") as f:
#         json.dump(original_convo, f, indent=2)

## Get activations for dialogues

- chat and not chat prompt formatted?

In [7]:
model, tokenizer = load_model(CHAT_MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

In [11]:
dialogue_activations = {}
for role, dialogue in dialogues.items():
    dialogue_activation = extract_full_activations(model, tokenizer, dialogue, chat_format=False)
    torch.save(dialogue_activations, f"{activations_dir}/dialogues/{role}.pt")

    dialogue_activations[role] = dialogue_activation

In [12]:
print(dialogue_activations["anxious_teenager"].shape)

torch.Size([46, 2589, 4608])


In [ ]:
# maybe could try base also

## Compare activations

In [ ]:
# could just mean the entire conversation?

# or just the responses of the assistant / 2nd interlocutor